# Module 4.5: K-Means Clustering

## 學習目標

完成這個 notebook 後，你將能夠：

1. 理解 K-Means 演算法的原理
2. 實作 K-Means 聚類
3. 實作 K-Means++ 初始化
4. 應用 K-Means 進行影像壓縮（顏色量化）

## 背景知識

K-Means 是最經典的**非監督式學習**演算法，用於將資料分成 K 個群組。

---

## 參考資源

- Bishop PRML Ch. 9.1: K-means Clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("環境設定完成！")

---

## Part 1: K-Means 演算法

### 目標

最小化 Within-Cluster Sum of Squares (WCSS)：

$$J = \sum_{k=1}^K \sum_{x_i \in C_k} \|x_i - \mu_k\|^2$$

其中 $\mu_k$ 是第 k 個 cluster 的中心。

### 演算法

1. **初始化**：隨機選擇 K 個中心
2. **重複直到收斂**：
   - **Assignment**：將每個點分配到最近的中心
   - **Update**：更新中心為該 cluster 所有點的平均

In [ ]:
# 生成測試資料
def generate_cluster_data(n_samples=300, n_clusters=3, seed=42):
    """
    生成聚類測試資料
    """
    np.random.seed(seed)
    
    n_per_cluster = n_samples // n_clusters
    
    # Cluster 中心
    centers = np.array([
        [0, 0],
        [4, 4],
        [8, 0]
    ])[:n_clusters]
    
    X_list = []
    y_list = []  # 真實標籤（用於評估）
    
    for k in range(n_clusters):
        X_k = np.random.randn(n_per_cluster, 2) * 1.0 + centers[k]
        X_list.append(X_k)
        y_list.append(np.full(n_per_cluster, k))
    
    X = np.vstack(X_list)
    y_true = np.concatenate(y_list)
    
    idx = np.random.permutation(len(y_true))
    return X[idx], y_true[idx]


# 視覺化
X, y_true = generate_cluster_data(n_samples=300, n_clusters=3)

plt.figure(figsize=(10, 8))
plt.scatter(X[:, 0], X[:, 1], c='gray', alpha=0.6, s=30)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('聚類測試資料（未標記）')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
class KMeans:
    """
    K-Means 聚類演算法
    
    Parameters
    ----------
    n_clusters : int
        聚類數量 K
    max_iter : int
        最大迭代次數
    init : str
        初始化方法：'random' 或 'kmeans++'
    tol : float
        收斂容差
    n_init : int
        隨機初始化次數（選最佳結果）
    """
    
    def __init__(self, n_clusters=3, max_iter=100, init='random', tol=1e-4, n_init=10):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.init = init
        self.tol = tol
        self.n_init = n_init
        
        self.centroids = None
        self.labels_ = None
        self.inertia_ = None  # WCSS
        self.n_iter_ = 0
    
    def _init_centroids_random(self, X):
        """
        隨機初始化：從資料中隨機選 K 個點
        """
        N = X.shape[0]
        indices = np.random.choice(N, self.n_clusters, replace=False)
        return X[indices].copy()
    
    def _init_centroids_kmeans_plus_plus(self, X):
        """
        K-Means++ 初始化
        
        1. 隨機選第一個中心
        2. 對於每個後續中心，選擇機率與距離平方成正比
        """
        N, D = X.shape
        centroids = np.zeros((self.n_clusters, D))
        
        # 第一個中心：隨機選
        centroids[0] = X[np.random.randint(N)]
        
        for k in range(1, self.n_clusters):
            # 計算每個點到最近中心的距離平方
            distances = np.zeros(N)
            for i in range(N):
                min_dist = np.inf
                for j in range(k):
                    dist = np.sum((X[i] - centroids[j]) ** 2)
                    if dist < min_dist:
                        min_dist = dist
                distances[i] = min_dist
            
            # 選擇機率與距離平方成正比
            probabilities = distances / np.sum(distances)
            centroids[k] = X[np.random.choice(N, p=probabilities)]
        
        return centroids
    
    def _assign_clusters(self, X):
        """
        Assignment step：將每個點分配到最近的中心
        """
        N = X.shape[0]
        labels = np.zeros(N, dtype=int)
        
        for i in range(N):
            distances = np.sum((self.centroids - X[i]) ** 2, axis=1)
            labels[i] = np.argmin(distances)
        
        return labels
    
    def _update_centroids(self, X, labels):
        """
        Update step：更新中心為群內平均
        """
        new_centroids = np.zeros_like(self.centroids)
        
        for k in range(self.n_clusters):
            mask = labels == k
            if np.sum(mask) > 0:
                new_centroids[k] = np.mean(X[mask], axis=0)
            else:
                # 空 cluster：保持原中心
                new_centroids[k] = self.centroids[k]
        
        return new_centroids
    
    def _compute_inertia(self, X, labels):
        """
        計算 WCSS (Within-Cluster Sum of Squares)
        """
        inertia = 0.0
        for k in range(self.n_clusters):
            mask = labels == k
            if np.sum(mask) > 0:
                inertia += np.sum((X[mask] - self.centroids[k]) ** 2)
        return inertia
    
    def _fit_single(self, X):
        """
        執行一次 K-Means
        """
        # 初始化
        if self.init == 'random':
            self.centroids = self._init_centroids_random(X)
        elif self.init == 'kmeans++':
            self.centroids = self._init_centroids_kmeans_plus_plus(X)
        else:
            raise ValueError(f"Unknown init method: {self.init}")
        
        # 迭代
        for iteration in range(self.max_iter):
            # Assignment
            labels = self._assign_clusters(X)
            
            # Update
            new_centroids = self._update_centroids(X, labels)
            
            # 檢查收斂
            shift = np.linalg.norm(new_centroids - self.centroids)
            self.centroids = new_centroids
            
            if shift < self.tol:
                break
        
        self.labels_ = labels
        self.inertia_ = self._compute_inertia(X, labels)
        self.n_iter_ = iteration + 1
        
        return self.inertia_
    
    def fit(self, X):
        """
        擬合 K-Means 模型
        
        執行 n_init 次，選擇 inertia 最小的結果
        """
        best_inertia = np.inf
        best_centroids = None
        best_labels = None
        
        for _ in range(self.n_init):
            inertia = self._fit_single(X)
            
            if inertia < best_inertia:
                best_inertia = inertia
                best_centroids = self.centroids.copy()
                best_labels = self.labels_.copy()
        
        self.centroids = best_centroids
        self.labels_ = best_labels
        self.inertia_ = best_inertia
        
        return self
    
    def predict(self, X):
        """
        預測新資料的 cluster 標籤
        """
        return self._assign_clusters(X)
    
    def fit_predict(self, X):
        """擬合並返回標籤"""
        self.fit(X)
        return self.labels_


# 測試
print("=== 測試 K-Means ===")

kmeans = KMeans(n_clusters=3, init='random', n_init=10)
labels = kmeans.fit_predict(X)

print(f"迭代次數: {kmeans.n_iter_}")
print(f"WCSS (Inertia): {kmeans.inertia_:.2f}")
print(f"Cluster 大小: {np.bincount(labels)}")

# 視覺化
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple', 'orange']
for k in range(kmeans.n_clusters):
    mask = labels == k
    plt.scatter(X[mask, 0], X[mask, 1], c=colors[k], s=30, 
               label=f'Cluster {k}', alpha=0.6)

# 繪製中心
plt.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1], 
           c='black', marker='X', s=200, label='Centroids')

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('K-Means 聚類結果')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 2: K-Means++ 初始化

隨機初始化可能導致：
1. 收斂到局部最優
2. 初始中心太近，聚類效果差

K-Means++ 改進：讓初始中心盡量分散。

In [ ]:
# 比較 Random vs K-Means++ 初始化
print("=== Random vs K-Means++ 初始化比較 ===")

# 多次運行比較
n_runs = 20
inertias_random = []
inertias_kpp = []

for _ in range(n_runs):
    # Random
    km_random = KMeans(n_clusters=3, init='random', n_init=1)
    km_random.fit(X)
    inertias_random.append(km_random.inertia_)
    
    # K-Means++
    km_kpp = KMeans(n_clusters=3, init='kmeans++', n_init=1)
    km_kpp.fit(X)
    inertias_kpp.append(km_kpp.inertia_)

print(f"Random 初始化 - Mean: {np.mean(inertias_random):.2f}, Std: {np.std(inertias_random):.2f}")
print(f"K-Means++ 初始化 - Mean: {np.mean(inertias_kpp):.2f}, Std: {np.std(inertias_kpp):.2f}")

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(inertias_random, bins=15, alpha=0.7, label='Random', color='blue')
axes[0].hist(inertias_kpp, bins=15, alpha=0.7, label='K-Means++', color='red')
axes[0].set_xlabel('Inertia (WCSS)')
axes[0].set_ylabel('Count')
axes[0].set_title('Inertia 分布比較')
axes[0].legend()

axes[1].boxplot([inertias_random, inertias_kpp], labels=['Random', 'K-Means++'])
axes[1].set_ylabel('Inertia (WCSS)')
axes[1].set_title('Inertia Box Plot')

plt.tight_layout()
plt.show()

print("\n觀察：K-Means++ 的 inertia 更低且更穩定")

---

## Part 3: Elbow Method 選擇 K

**問題**：如何選擇最佳的 K 值？

**Elbow Method**：繪製 K vs Inertia 曲線，找「肘點」。

In [ ]:
# Elbow Method
print("=== Elbow Method ===")

K_range = range(1, 10)
inertias = []

for K in K_range:
    km = KMeans(n_clusters=K, init='kmeans++', n_init=10)
    km.fit(X)
    inertias.append(km.inertia_)
    print(f"K={K}: Inertia = {km.inertia_:.2f}")

# 視覺化
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia (WCSS)')
plt.title('Elbow Method for Optimal K')
plt.xticks(K_range)
plt.grid(True, alpha=0.3)

# 標記肘點
plt.axvline(x=3, color='r', linestyle='--', label='Elbow at K=3')
plt.legend()
plt.show()

print("\n觀察：K=3 後 inertia 下降變緩，表示 K=3 是好的選擇")

---

## Part 4: 應用 - 影像顏色量化

使用 K-Means 將影像的顏色數量減少到 K 種。

In [ ]:
# 建立測試圖像（漸層）
def create_gradient_image(height=100, width=100):
    """建立彩色漸層圖像"""
    img = np.zeros((height, width, 3), dtype=np.uint8)
    
    for y in range(height):
        for x in range(width):
            img[y, x, 0] = int(255 * x / width)      # R: 左到右
            img[y, x, 1] = int(255 * y / height)     # G: 上到下
            img[y, x, 2] = int(255 * (1 - x/width))  # B: 右到左
    
    return img


def color_quantization(image, n_colors):
    """
    使用 K-Means 進行顏色量化
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W, 3)
        RGB 圖像
    n_colors : int
        目標顏色數量
    
    Returns
    -------
    quantized : np.ndarray
        量化後的圖像
    """
    H, W, C = image.shape
    
    # 將圖像展平為 (N, 3) 的點集
    pixels = image.reshape(-1, 3).astype(np.float64)
    
    # 執行 K-Means
    kmeans = KMeans(n_clusters=n_colors, init='kmeans++', n_init=3, max_iter=50)
    labels = kmeans.fit_predict(pixels)
    
    # 用中心顏色替換每個像素
    quantized_pixels = kmeans.centroids[labels]
    
    # 重塑為圖像
    quantized = quantized_pixels.reshape(H, W, C).astype(np.uint8)
    
    return quantized, kmeans.centroids


# 測試
print("=== 影像顏色量化 ===")

# 建立測試圖像
img = create_gradient_image(100, 100)

# 不同的 K 值
K_values = [2, 4, 8, 16]

fig, axes = plt.subplots(1, len(K_values) + 1, figsize=(16, 4))

# 原圖
axes[0].imshow(img)
axes[0].set_title(f'Original\n({256**3} colors possible)')
axes[0].axis('off')

# 量化結果
for i, K in enumerate(K_values):
    quantized, palette = color_quantization(img, K)
    axes[i+1].imshow(quantized)
    axes[i+1].set_title(f'K={K} colors')
    axes[i+1].axis('off')

plt.suptitle('K-Means 顏色量化', fontsize=14)
plt.tight_layout()
plt.show()

# 顯示調色板
print("\n顯示 K=8 的調色板：")
quantized_8, palette_8 = color_quantization(img, 8)

fig, ax = plt.subplots(figsize=(10, 2))
palette_display = np.zeros((50, 400, 3), dtype=np.uint8)
for i, color in enumerate(palette_8):
    palette_display[:, i*50:(i+1)*50] = color.astype(np.uint8)

ax.imshow(palette_display)
ax.set_title('Color Palette (K=8)')
ax.axis('off')
plt.show()

---

## 練習題

### 練習 1：向量化的 K-Means

In [ ]:
# 練習 1 解答：向量化實作

class KMeansVectorized:
    """
    向量化的 K-Means（更快）
    """
    
    def __init__(self, n_clusters=3, max_iter=100, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None
        self.labels_ = None
        self.inertia_ = None
    
    def _compute_distances(self, X):
        """
        向量化計算每個點到每個中心的距離
        
        ||x - c||² = ||x||² + ||c||² - 2*x·c
        """
        # X: (N, D), centroids: (K, D)
        # 結果: (N, K)
        
        # ||x||²: shape (N, 1)
        X_sq = np.sum(X ** 2, axis=1, keepdims=True)
        
        # ||c||²: shape (1, K)
        C_sq = np.sum(self.centroids ** 2, axis=1, keepdims=True).T
        
        # x·c: shape (N, K)
        XC = X @ self.centroids.T
        
        # ||x - c||²
        distances = X_sq + C_sq - 2 * XC
        
        return distances
    
    def _assign_clusters(self, X):
        """向量化 assignment"""
        distances = self._compute_distances(X)
        return np.argmin(distances, axis=1)
    
    def _update_centroids(self, X, labels):
        """向量化 update"""
        new_centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            mask = labels == k
            if np.sum(mask) > 0:
                new_centroids[k] = np.mean(X[mask], axis=0)
            else:
                new_centroids[k] = self.centroids[k]
        return new_centroids
    
    def fit(self, X):
        N = X.shape[0]
        
        # K-Means++ 初始化
        indices = [np.random.randint(N)]
        for k in range(1, self.n_clusters):
            self.centroids = X[indices]
            distances = self._compute_distances(X)
            min_distances = np.min(distances, axis=1)
            probs = min_distances / np.sum(min_distances)
            indices.append(np.random.choice(N, p=probs))
        
        self.centroids = X[indices]
        
        # 迭代
        for _ in range(self.max_iter):
            labels = self._assign_clusters(X)
            new_centroids = self._update_centroids(X, labels)
            
            shift = np.linalg.norm(new_centroids - self.centroids)
            self.centroids = new_centroids
            
            if shift < self.tol:
                break
        
        self.labels_ = labels
        distances = self._compute_distances(X)
        self.inertia_ = np.sum(distances[np.arange(N), labels])
        
        return self
    
    def predict(self, X):
        return self._assign_clusters(X)


# 比較速度
import time

# 生成較大的資料集
X_large = np.random.randn(5000, 10)

print("=== 速度比較 ===")

# 原始版本
start = time.time()
km1 = KMeans(n_clusters=5, n_init=1, max_iter=50, init='kmeans++')
km1.fit(X_large)
time1 = time.time() - start
print(f"原始版本: {time1:.3f}s, Inertia: {km1.inertia_:.2f}")

# 向量化版本
start = time.time()
km2 = KMeansVectorized(n_clusters=5, max_iter=50)
km2.fit(X_large)
time2 = time.time() - start
print(f"向量化版本: {time2:.3f}s, Inertia: {km2.inertia_:.2f}")

print(f"\n加速比: {time1/time2:.2f}x")

### 練習 2：K-Means 動畫

In [ ]:
# 練習 2 解答：視覺化 K-Means 收斂過程

def kmeans_step_by_step(X, n_clusters=3, max_iter=10):
    """
    執行 K-Means 並記錄每一步的狀態
    """
    N, D = X.shape
    
    # 初始化（隨機選擇，方便展示）
    np.random.seed(0)
    indices = np.random.choice(N, n_clusters, replace=False)
    centroids = X[indices].copy()
    
    history = []
    
    for i in range(max_iter):
        # Assignment
        distances = np.sum((X[:, np.newaxis, :] - centroids[np.newaxis, :, :]) ** 2, axis=2)
        labels = np.argmin(distances, axis=1)
        
        # 記錄
        history.append({
            'iteration': i,
            'centroids': centroids.copy(),
            'labels': labels.copy()
        })
        
        # Update
        new_centroids = np.array([X[labels == k].mean(axis=0) if np.sum(labels == k) > 0 else centroids[k] 
                                  for k in range(n_clusters)])
        
        # 檢查收斂
        if np.allclose(new_centroids, centroids):
            break
        
        centroids = new_centroids
    
    return history


# 執行並視覺化
X, _ = generate_cluster_data(n_samples=200, n_clusters=3, seed=42)
history = kmeans_step_by_step(X, n_clusters=3, max_iter=10)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

colors = ['blue', 'red', 'green']

for idx, state in enumerate(history[:6]):
    ax = axes[idx]
    
    # 繪製點
    for k in range(3):
        mask = state['labels'] == k
        ax.scatter(X[mask, 0], X[mask, 1], c=colors[k], s=30, alpha=0.6)
    
    # 繪製中心
    ax.scatter(state['centroids'][:, 0], state['centroids'][:, 1], 
              c='black', marker='X', s=200)
    
    ax.set_title(f"Iteration {state['iteration']}")
    ax.grid(True, alpha=0.3)

plt.suptitle('K-Means 收斂過程', fontsize=14)
plt.tight_layout()
plt.show()

---

## 總結

### 本 Notebook 涵蓋的內容

1. **K-Means 演算法**：
   - Assignment + Update 迭代
   - 最小化 WCSS

2. **K-Means++ 初始化**：
   - 讓初始中心盡量分散
   - 更穩定、更好的結果

3. **Elbow Method**：
   - 選擇最佳 K 值
   - 找 WCSS 曲線的「肘點」

4. **應用**：
   - 影像顏色量化
   - 資料壓縮

### 關鍵要點

1. K-Means 是 hard clustering（每個點只屬於一個 cluster）
2. 結果依賴初始化
3. K 需要預先指定

### 下一步

在下一個 notebook 中，我們將學習 **GMM + EM**（軟聚類）。